In [16]:
import os
import pandas as pd
import numpy as np
import urllib

In [17]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC'
normpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\normcomp'
maxpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\maximums\maxcomp'
minpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\minimums\mincomp'

os.chdir(path)

xl_file = r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\colocation_metadata.xlsx"
csv = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\COOP_Normals\csv.csv'

xl = pd.ExcelFile(xl_file)

meta = xl.parse('metadata')
meta['sdate'] = meta.sdate.dt.date
meta['edate'] = meta.edate.dt.date
meta.drop(meta.columns[3],axis=1)

n = 0
for index, rows in meta.iterrows():
    print('Grabbing CoAg data for ' + str(meta.ID[n]))
    urllib.request.urlretrieve(
        'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=' + str(meta.ID[n]) + '&sdate=' + str(meta.sdate[n]) + '&edate=' + str(meta.edate[n]) + '&elems=tave,tmax,tmin,vp,rhmax,rhmin,sr,wrun,pp,st5mx,st5mn,st15mx,st15mn',
        filename=csv)
    coag = pd.read_csv(csv)
    print('Cleaning CoAg data for ' + str(meta.ID[n]))
    coag = coag.reset_index()
    coag.columns.values[0] = 'station'
    coag.columns.values[1] = 'date'
    coag.columns.values[2] = 'CoAg_tmean'
    coag.columns.values[3] = 'CoAg_tmax'
    coag.columns.values[4] = 'CoAg_tmin'
    coag.columns.values[5] = 'vp'
    coag.columns.values[6] = 'rhmax'
    coag.columns.values[7] = 'rhmin'
    coag.columns.values[8] = 'sr'
    coag.columns.values[9] = 'wrun'
    coag.columns.values[10] = 'pp'
    coag.columns.values[11] = 'st5mx'
    coag.columns.values[12] = 'st5mn'
    coag.columns.values[13] = 'st15mx'
    coag.columns.values[14] = 'st15mn'
    
    stuff = ['station','vp','rhmax','rhmin','sr','wrun','pp','st5mx','st5mn','st15mx','st15mn']
    coag = coag.drop(stuff, axis=1)
    
    coag['date'] = pd.to_datetime(coag.date)
    coag['julian'] = coag.date.dt.strftime('%j')
    
    coag.CoAg_tmean = 32 + (coag.CoAg_tmean * (9/5))
    coag.CoAg_tmax = 32 + (coag.CoAg_tmax * (9/5))
    coag.CoAg_tmin = 32 + (coag.CoAg_tmin * (9/5))
    
    coag = coag[:-1]
    
    print('Calculating CoAg normals for ' + str(meta.ID[n]))
    coag_norm = coag.groupby(['julian']).mean().reset_index()
    coag_max = coag.groupby(['julian']).max().reset_index()
    coag_min = coag.groupby(['julian']).min().reset_index()
    
    #coag_norm_res = coag.merge(coag_norm,'outer',left_on='julian',right_on='julian',suffixes=('_observed','_normal'))
    #coag_norm_res['CoAg_tmean_res'] = coag_norm_res['CoAg_tmean_normal'] - coag_norm_res['CoAg_tmean_observed']
    #coag_norm_res['CoAg_tmax_res'] = coag_norm_res['CoAg_tmax_normal'] - coag_norm_res['CoAg_tmax_observed']
    #coag_norm_res['CoAg_tmin_res'] = coag_norm_res['CoAg_tmin_normal'] - coag_norm_res['CoAg_tmin_observed']
    #coag_norm_res = coag_norm_res[['date'],['CoAg_tmean'],['CoAg_tmax'],['CoAg_tmin'],[]]
    
    #coag_max_res = coag.merge(coag_max,'outer',left_on='julian',right_on='julian',suffixes=('_observed','_normal'))
    #coag_min_res = coag.merge(coag_min,'outer',left_on='julian',right_on='julian',suffixes=('_observed','_normal'))
                                   
    
    print('Grabbing COOP data for ' + str(meta.ID[n]))
    urllib.request.urlretrieve(
        'http://data.rcc-acis.org/StnData?sid=0' + str(meta.COOP_ID[n]) + '&sdate='+ str(meta.sdate[n]) + '&edate=' + str(meta.edate[n]) + '&elems=maxt,mint,avgt,pcpn&output=csv',
        filename=csv)
    print('Cleaning COOP data for ' + str(meta.ID[n]))
    coop = pd.read_csv(csv)
    coop = coop.reset_index()
    coop.columns.values[0] = 'date'
    coop.columns.values[1] = 'COOP_tmax'
    coop.columns.values[2] = 'COOP_tmin'
    coop.columns.values[3] = 'COOP_tmean'
    coop.columns.values[4] = 'pcpn'
    coop = coop.drop('pcpn', axis=1)
    
    coop['date'] = pd.to_datetime(coop.date)
    coop['julian'] = coop.date.dt.strftime('%j')
    
    coop = coop.replace('M', np.nan, regex=True)
    coop = coop.replace('T', 0.005, regex=True)
   
    coop.COOP_tmax = coop.COOP_tmax.astype(float)
    coop.COOP_tmin = coop.COOP_tmin.astype(float)
    coop.COOP_tmean = coop.COOP_tmean.astype(float)
    print('Calculating COOP normals for ' + str(meta.ID[n]))
    
    coop_norm = coop.groupby(['julian']).mean().reset_index()
    coop_max = coop.groupby(['julian']).max().reset_index()
    coop_min = coop.groupby(['julian']).min().reset_index()
    print('Merging COOP and CoAg data for ' + str(meta.ID[n]))
    df_norm = pd.merge(coag_norm,coop_norm, on=coag_norm.julian)
    df_max = pd.merge(coag_max,coop_max,on=coag_max.julian)
    df_min = pd.merge(coag_min,coop_min,on=coag_min.julian)
    
    stuff = ['key_0','julian_y']

    df_norm = df_norm.drop(stuff, axis=1)
    df_max = df_max.drop(stuff, axis=1)
    df_min = df_min.drop(stuff, axis=1)

    df_norm = df_norm[['julian_x','CoAg_tmean','COOP_tmean','CoAg_tmax','COOP_tmax','CoAg_tmin','COOP_tmin']]
    df_max = df_max[['julian_x','CoAg_tmean','COOP_tmean','CoAg_tmax','COOP_tmax','CoAg_tmin','COOP_tmin']]
    df_min = df_min[['julian_x','CoAg_tmean','COOP_tmean','CoAg_tmax','COOP_tmax','CoAg_tmin','COOP_tmin']]

    df_norm.columns.values[0] = 'julian_day'
    df_max.columns.values[0] = 'julian_day'
    df_min.columns.values[0] = 'julian_day'
    
    df_norm = df_norm.round(2)
    df_max = df_max.round(2)
    df_min = df_min.round(2)
    
    
    print('Creating csv files of the merged datasets for ' + str(meta.ID[n]))
    df_norm.to_csv(normpath + r'/' + str(meta.ID[n]) + '_normComp.csv')
    df_max.to_csv(maxpath + r'/' + str(meta.ID[n]) +'_maxComp.csv')
    df_min.to_csv(minpath + r'/' + str(meta.ID[n]) + '_minComp.csv')
     
    n = n + 1

Grabbing CoAg data for AKR02
Cleaning CoAg data for AKR02
Calculating CoAg normals for AKR02
Converting CoAg temps from C° to F°  for AKR02
Grabbing COOP data for AKR02
Cleaning COOP data for AKR02
Calculating COOP normals for AKR02
Merging COOP and CoAg data for AKR02
Creating csv files of the merged datasets for AKR02
Grabbing CoAg data for ALT01
Cleaning CoAg data for ALT01
Calculating CoAg normals for ALT01
Converting CoAg temps from C° to F°  for ALT01
Grabbing COOP data for ALT01
Cleaning COOP data for ALT01
Calculating COOP normals for ALT01
Merging COOP and CoAg data for ALT01
Creating csv files of the merged datasets for ALT01
Grabbing CoAg data for AVN01
Cleaning CoAg data for AVN01
Calculating CoAg normals for AVN01
Converting CoAg temps from C° to F°  for AVN01
Grabbing COOP data for AVN01
Cleaning COOP data for AVN01
Calculating COOP normals for AVN01
Merging COOP and CoAg data for AVN01
Creating csv files of the merged datasets for AVN01
Grabbing CoAg data for BNV01
Clean

Cleaning COOP data for GUN01
Calculating COOP normals for GUN01
Merging COOP and CoAg data for GUN01
Creating csv files of the merged datasets for GUN01
Grabbing CoAg data for HEB01
Cleaning CoAg data for HEB01
Calculating CoAg normals for HEB01
Converting CoAg temps from C° to F°  for HEB01
Grabbing COOP data for HEB01
Cleaning COOP data for HEB01
Calculating COOP normals for HEB01
Merging COOP and CoAg data for HEB01
Creating csv files of the merged datasets for HEB01
Grabbing CoAg data for HLY01
Cleaning CoAg data for HLY01
Calculating CoAg normals for HLY01
Converting CoAg temps from C° to F°  for HLY01
Grabbing COOP data for HLY01
Cleaning COOP data for HLY01
Calculating COOP normals for HLY01
Merging COOP and CoAg data for HLY01
Creating csv files of the merged datasets for HLY01
Grabbing CoAg data for HLY02
Cleaning CoAg data for HLY02
Calculating CoAg normals for HLY02
Converting CoAg temps from C° to F°  for HLY02
Grabbing COOP data for HLY02
Cleaning COOP data for HLY02
Calcu

Cleaning CoAg data for ORM01
Calculating CoAg normals for ORM01
Converting CoAg temps from C° to F°  for ORM01
Grabbing COOP data for ORM01
Cleaning COOP data for ORM01
Calculating COOP normals for ORM01
Merging COOP and CoAg data for ORM01
Creating csv files of the merged datasets for ORM01
Grabbing CoAg data for OTH01
Cleaning CoAg data for OTH01
Calculating CoAg normals for OTH01
Converting CoAg temps from C° to F°  for OTH01
Grabbing COOP data for OTH01
Cleaning COOP data for OTH01
Calculating COOP normals for OTH01
Merging COOP and CoAg data for OTH01
Creating csv files of the merged datasets for OTH01
Grabbing CoAg data for PAI01
Cleaning CoAg data for PAI01
Calculating CoAg normals for PAI01
Converting CoAg temps from C° to F°  for PAI01
Grabbing COOP data for PAI01
Cleaning COOP data for PAI01
Calculating COOP normals for PAI01
Merging COOP and CoAg data for PAI01
Creating csv files of the merged datasets for PAI01
Grabbing CoAg data for PKH01
Cleaning CoAg data for PKH01
Calcu